In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt 

C:\Users\shonn\AppData\Local\Temp\ipykernel_20564\2307468532.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# 데이터 불러오기

In [2]:
# 날씨데이터
df_weather = pd.read_csv('weather_treatment.csv')

# 관중데이터
df_spec = pd.read_csv('spectator.csv')

# KBO 경기결과 데이터
df_kbo = pd.read_csv('kbo_winlose_final.csv')

### 날씨-관중 데이터 합치기

> 날씨데이터 전처리

In [3]:
# 날짜데이터 - 하이픈 삭제

df_weather['일시'] = df_weather['일시'].str.replace('-', '')
df_weather

,Unnamed: 0,지점,지점명,일시,평균기온,최저기온,최고기온,일강수량,풍속,습도,강수여부
0,0,108,서울,20210101,-4.2,-9.8,1.6,0.0,2.0,64.0,sunny
1,1,108,서울,20210102,-5.0,-8.4,-1.4,0.0,2.6,38.5,sunny
2,2,108,서울,20210103,-5.6,-9.1,-2.0,0.0,2.0,45.0,sunny
3,3,108,서울,20210104,-3.5,-8.4,0.3,0.0,1.7,51.4,sunny
4,4,108,서울,20210105,-5.5,-9.9,-2.1,0.0,2.9,52.8,sunny
...,...,...,...,...,...,...,...,...,...,...,...
11815,11815,159,부산,20240323,16.1,12.3,20.5,0.7,5.2,73.6,rainy
11816,11816,159,부산,20240324,11.3,10.1,13.6,3.7,4.2,77.5,rainy
11817,11817,159,부산,20240325,10.5,8.7,12.4,9.4,4.2,82.4,rainy
11818,11818,159,부산,20240326,8.8,6.2,12.9,9.8,3.8,75.4,rainy


In [5]:

# 원하는 정보만 남기기

cond_w = ['지점명', '일시', '평균기온', '일강수량', '풍속', '습도', '강수여부']
df_weather2 = df_weather[cond]
df_weather2


,지점명,일시,평균기온,일강수량,풍속,습도,강수여부
0,서울,20210101,-4.2,0.0,2.0,64.0,sunny
1,서울,20210102,-5.0,0.0,2.6,38.5,sunny
2,서울,20210103,-5.6,0.0,2.0,45.0,sunny
3,서울,20210104,-3.5,0.0,1.7,51.4,sunny
4,서울,20210105,-5.5,0.0,2.9,52.8,sunny
...,...,...,...,...,...,...,...
11815,부산,20240323,16.1,0.7,5.2,73.6,rainy
11816,부산,20240324,11.3,3.7,4.2,77.5,rainy
11817,부산,20240325,10.5,9.4,4.2,82.4,rainy
11818,부산,20240326,8.8,9.8,3.8,75.4,rainy


> 관중데이터 전처리

In [6]:
# 년월일 두자리수 맞추고 str형태로 변경

df_spec['월'] = df_spec['월'].apply(lambda x : str(x).zfill(2))
df_spec['일'] = df_spec['일'].apply(lambda x : str(x).zfill(2))
df_spec['연도'] = df_spec['연도'].astype(str)

# 일시 컬럼 생성

df_spec['일시'] = df_spec['연도'] + df_spec['월'] + df_spec['일']
df_spec

# 홈팀 원정팀 (기아/SSG 팀명 변경)
df_spec['원정팀'] = df_spec['원정팀'].str.replace("KIA", "기아", 1)
df_spec['홈팀'] = df_spec['홈팀'].str.replace("KIA", "기아", 1)
df_spec['원정팀'] = df_spec['원정팀'].str.replace("SSG", "SK", 1)
df_spec['홈팀'] = df_spec['홈팀'].str.replace("SSG", "SK", 1)
df_spec['원정팀'] = df_spec['원정팀'].str.replace("키움히어로즈", "키움", 1)
df_spec['홈팀'] = df_spec['홈팀'].str.replace("키움히어로즈", "키움", 1)
df_spec



# 경기코드 생성
def gamecode(data) : 
        data['code'] = data['일시'] + data['원정팀'].str[:1] + data['홈팀'].str[:1]

gamecode(df_spec)
df_spec


# 지점명 생성
df_spec['지점명'] = df_spec['경기장'].str[:2]
df_spec

,연도,월,일,요일,홈팀,원정팀,경기장,날씨,평균기온,관중수,일시,code,지점명
0,2023,10,17,화,기아,NC,광주-기아 챔피언스 필드,맑음,21.7,"10,175",20231017,20231017N기,광주
1,2023,10,17,화,SK,두산,인천SSG 랜더스필드,맑음,19.5,"21,007",20231017,20231017두S,인천
2,2023,10,16,월,두산,SK,서울 잠실 야구장,맑음,19.6,"15,850",20231016,20231016S두,서울
3,2023,10,16,월,한화,롯데,대전 한화생명 이글스파크,맑음,20.7,"12,000",20231016,20231016롯한,대전
4,2023,10,16,월,기아,NC,광주-기아 챔피언스 필드,맑음,20.6,"5,251",20231016,20231016N기,광주
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2433,2021,04,03,토,두산,기아,서울 잠실 야구장,비,18.0,0,20210403,20210403기두,서울
2434,2021,04,03,토,키움,삼성,서울고척스카이돔,비,18.0,1665,20210403,20210403삼키,서울
2435,2021,04,03,토,KT,한화,수원케이티위즈파크,비,18.6,0,20210403,20210403한K,수원
2436,2021,04,03,토,NC,LG,창원NC파크,비,17.0,0,20210403,20210403LN,창원


In [8]:

# 원하는 데이터만 남기기

cond_s = ['일시', '요일', '홈팀', '원정팀', '경기장', '지점명', '관중수', 'code']
df_spec2 = df_spec[cond_s]
df_spec2


,일시,요일,홈팀,원정팀,경기장,지점명,관중수,code
0,20231017,화,기아,NC,광주-기아 챔피언스 필드,광주,"10,175",20231017N기
1,20231017,화,SK,두산,인천SSG 랜더스필드,인천,"21,007",20231017두S
2,20231016,월,두산,SK,서울 잠실 야구장,서울,"15,850",20231016S두
3,20231016,월,한화,롯데,대전 한화생명 이글스파크,대전,"12,000",20231016롯한
4,20231016,월,기아,NC,광주-기아 챔피언스 필드,광주,"5,251",20231016N기
...,...,...,...,...,...,...,...,...
2433,20210403,토,두산,기아,서울 잠실 야구장,서울,0,20210403기두
2434,20210403,토,키움,삼성,서울고척스카이돔,서울,1665,20210403삼키
2435,20210403,토,KT,한화,수원케이티위즈파크,수원,0,20210403한K
2436,20210403,토,NC,LG,창원NC파크,창원,0,20210403LN


#### 날씨-관중 데이터 병합하기

In [12]:
# 날씨데이터  df_weather2
# 관중데이터  df_spec2
# 관중데이터로 left join / on = 일시

weather_spec = pd.merge(left = df_weather2, right = df_spec2, how = 'right', on = ['일시', '지점명'])

# 일시 기준으로 내림차순

weather_spec = weather_spec.sort_values(by=['일시'], ascending= True)
weather_spec.reset_index(drop = True, inplace = True)
weather_spec


# 필요한 데이터만 추출하기
cond_ws = ['code', '평균기온', '일강수량', '풍속', '습도', '경기장', '지점명']
weather_spec = weather_spec[cond_ws]
weather_spec

,code,평균기온,일강수량,풍속,습도,경기장,지점명
0,20210403롯S,14.1,54.7,3.1,79.6,인천SSG 랜더스필드,인천
1,20210403기두,14.9,56.2,3.1,80.8,서울 잠실 야구장,서울
2,20210403LN,15.4,38.7,1.0,84.8,창원NC파크,창원
3,20210403한K,15.4,46.7,1.9,85.4,수원케이티위즈파크,수원
4,20210403삼키,14.9,56.2,3.1,80.8,서울고척스카이돔,서울
...,...,...,...,...,...,...,...
2433,20231016N기,15.8,0.0,0.9,77.8,광주-기아 챔피언스 필드,광주
2434,20231016롯한,14.7,0.0,1.7,64.5,대전 한화생명 이글스파크,대전
2435,20231016S두,14.7,0.0,2.4,57.0,서울 잠실 야구장,서울
2436,20231017두S,14.8,0.0,1.6,50.8,인천SSG 랜더스필드,인천


## 관중-날씨데이터 & 승리 데이터 합치기

In [13]:
# 왼쪽 weather_spec
# 오른쪽 df_kbo2 

eda_data = pd.merge(left = weather_spec, right = df_kbo, how = 'right', on = 'code')
eda_data


,code,평균기온,일강수량,풍속,습도,경기장,지점명,Unnamed: 0,date,week,원정팀,홈팀,A,B,승리팀,승리팀구분
0,20210403기두,14.9,56.2,3.1,80.8,서울 잠실 야구장,서울,0,20210403,토,기아,두산,NaN,NaN,우천취소,Away
1,20210403롯S,14.1,54.7,3.1,79.6,인천SSG 랜더스필드,인천,1,20210403,토,롯데,SK,NaN,NaN,우천취소,Away
2,20210403LN,15.4,38.7,1.0,84.8,창원NC파크,창원,2,20210403,토,LG,NC,NaN,NaN,우천취소,Away
3,20210403한K,15.4,46.7,1.9,85.4,수원케이티위즈파크,수원,3,20210403,토,한화,KT,NaN,NaN,우천취소,Away
4,20210403삼키,14.9,56.2,3.1,80.8,서울고척스카이돔,서울,4,20210403,토,삼성,키움,1.0,6.0,키움,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396,20231016S두,14.7,0.0,2.4,57.0,서울 잠실 야구장,서울,808,20231016,월,SK,두산,3.0,2.0,SK,Away
2397,20231016N기,15.8,0.0,0.9,77.8,광주-기아 챔피언스 필드,광주,809,20231016,월,NC,기아,2.0,4.0,기아,Home
2398,20231016롯한,14.7,0.0,1.7,64.5,대전 한화생명 이글스파크,대전,810,20231016,월,롯데,한화,7.0,2.0,롯데,Away
2399,20231017두S,14.8,0.0,1.6,50.8,인천SSG 랜더스필드,인천,811,20231017,화,두산,SK,0.0,5.0,SK,Home


In [26]:
cond_eda = ['code', '원정팀', '홈팀', 'A', 'B', '승리팀', '승리팀구분', 'date', 'week', '평균기온', '일강수량', '풍속', '습도', '경기장', '지점명']
final_df = eda_data[cond_eda]

final_df['date'] = final_df['date'].astype(str)
final_df['year'] = final_df['date'].str[:4]

final_df

C:\Users\shonn\AppData\Local\Temp\ipykernel_20564\3902061046.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['date'] = final_df['date'].astype(str)


,code,원정팀,홈팀,A,B,승리팀,승리팀구분,date,week,평균기온,일강수량,풍속,습도,경기장,지점명,year
0,20210403기두,기아,두산,NaN,NaN,우천취소,Away,20210403,토,14.9,56.2,3.1,80.8,서울 잠실 야구장,서울,2021
1,20210403롯S,롯데,SK,NaN,NaN,우천취소,Away,20210403,토,14.1,54.7,3.1,79.6,인천SSG 랜더스필드,인천,2021
2,20210403LN,LG,NC,NaN,NaN,우천취소,Away,20210403,토,15.4,38.7,1.0,84.8,창원NC파크,창원,2021
3,20210403한K,한화,KT,NaN,NaN,우천취소,Away,20210403,토,15.4,46.7,1.9,85.4,수원케이티위즈파크,수원,2021
4,20210403삼키,삼성,키움,1.0,6.0,키움,Home,20210403,토,14.9,56.2,3.1,80.8,서울고척스카이돔,서울,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396,20231016S두,SK,두산,3.0,2.0,SK,Away,20231016,월,14.7,0.0,2.4,57.0,서울 잠실 야구장,서울,2023
2397,20231016N기,NC,기아,2.0,4.0,기아,Home,20231016,월,15.8,0.0,0.9,77.8,광주-기아 챔피언스 필드,광주,2023
2398,20231016롯한,롯데,한화,7.0,2.0,롯데,Away,20231016,월,14.7,0.0,1.7,64.5,대전 한화생명 이글스파크,대전,2023
2399,20231017두S,두산,SK,0.0,5.0,SK,Home,20231017,화,14.8,0.0,1.6,50.8,인천SSG 랜더스필드,인천,2023
